# Auslesen der Management-Transaktionen bei SIX

 Diese Anwendung basiert logisch auf "Selenium auf SIX Management Transaktionen". Ziel ist jedoch, die einzelne Selenium-Abfrage in eine funktion zu Packen, die dann wiederum in Schleifen aufgerufen werden kann.
 
 Die Idee ist, die Abfrage in Selenium und die Auswertung in separate Funktionen auszulagern und diese am ende mit einem kurzen Code automatisiert zu starten. 
 
 Dabei wird erst die Funktion selenium() aufgerufen. Das Ergebnis geht dann an die Funktion suppe(), welche ihrerseits wiederum in einer Schleife die Funktion meldungen() einsetzt.
 
 Aktuelles Problem: Obwohl Selenium die richtigen Daten setzt und die Abfragen auch die richtigen Daten liefert, erscheinen am Ende Werte aus einem anderen Monat im Endergebnis. Da dies bei getrennten Einzelausführungen nicht der Fall ist, muss der Fehler am Code liegen und nicht an der Website.

## Allgemeine Vorbereitung

In [2]:
#Importe

from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import os

import pandas as PD
%matplotlib inline

# Alles in eine Funktion werfen

In [ ]:
#def selenium(startdatum, enddatum):
#definition webdriver (ohne Pfadangabe, da Pfad gespeichert)

driver = webdriver.Chrome()

#definition Website
driver.get("https://www.six-exchange-regulation.com/de/home/publications/management-transactions.html")
#driver.get("https://www.six-exchange-regulation.com/de/home/publications/management-transactions.html#notificationId=T1I2G00075")

#time.sleep(1)           

#datum setzen
startdatum = "01012018"
enddatum = "20012018"

datum_von = driver.find_element_by_id('FromTransactionDate')
datum_von.send_keys(startdatum)
datum_bis = driver.find_element_by_name('ToTransactionDate')
datum_bis.send_keys(enddatum)       

#Einfach nur Abschicken
datum_von = driver.find_element_by_id('FromTransactionDate')
datum_von.send_keys(Keys.ENTER)

time.sleep(10)   

#HTML-Inhalt erfassen und übergeben
html = driver.page_source
html


 # Selenium-Abfrage als Funktion
 
 Die Funktion soll nach Eingabe eines Start- und Enddatums einen HTML-Code des Seite ausgeben.

In [14]:
#Ab der SIX-Seite für ein bestimmtes Datum alle meldungen auslesen und als HTML speichern.
#Ausgabe ist website als html. 

def selenium(startdatum, enddatum):
    #definition webdriver (ohne Pfadangabe, da Pfad gespeichert)
    driver = webdriver.Chrome()
    
    #definition Website
    driver.get("https://www.six-exchange-regulation.com/de/home/publications/management-transactions.html")
               
    #datum setzen
    datum_von = driver.find_element_by_id('FromTransactionDate')
    datum_von.send_keys(startdatum)          
    datum_bis = driver.find_element_by_name('ToTransactionDate')
    datum_bis.send_keys(enddatum)
               
    time.sleep(1)           
    
    #Alle Firmen wählen
    #select = Select(driver.find_element_by_name("NotificationSubmitterId"))
    #select.select_by_value("") #Für alle Aktien
    
    #Einfach nur Abschicken
    datum_von = driver.find_element_by_id('FromTransactionDate')
    datum_von.send_keys(Keys.ENTER)
    
    time.sleep(10)   
    
    #HTML-Inhalt erfassen und übergeben
    html = driver.page_source

    return html 

In [15]:
#test
liste=[]

start = "01012018"
ende = "20012018"

selenium(start, ende)

# Suppe als Funktion

Diese Funktion soll den über Selenium() generierten Code auswerten und als Liste ausgeben. 

## Aus der Website die einzelnen Zeilen auslesen

In [10]:
def suppe(site):
    liste = []
    website = str(site)
    soup = BeautifulSoup(website, 'lxml')
    ergebniszeilen_gerade = soup.find_all("tr", class_="itf-mark-row-template even")
    ergebniszeilen_ungerade = soup.find_all("tr", class_="itf-mark-row-template odd")
    allezeilen = ergebniszeilen_gerade + ergebniszeilen_ungerade
    
   
    for meldung in ergebniszeilen_ungerade:
        meldung = str(meldung)
        meldungen(meldung)

    for meldung in ergebniszeilen_gerade:
        meldung = str(meldung)
        meldungen(meldung)

    return liste

## Aus den extrahierten Meldungen die Daten auslesen

Diese Funktion ist eine Unterfunktion von suppe(), die von suppe() in einer Schleife aufgerufen wird.

In [6]:
#Aus einer Datenzeile die Meldungen extrahieren

def meldungen(zeile):

    soup_zeile=BeautifulSoup(zeile, 'lxml')

    datum = soup_zeile.select('[data-itf-inject]')[0].text
    unternehmen = soup_zeile.select('[data-itf-inject]')[1].text
    art = soup_zeile.select('[data-itf-inject]')[2].text
    #anzahl = float(soup_zeile.select('[data-itf-inject]')[3].text.replace("'",""))
    anzahl = soup_zeile.select('[data-itf-inject]')[3].text.replace("'","")
    wertpapier = soup_zeile.select('[data-itf-inject]')[4].text
    #wert = float(soup_zeile.select('[data-itf-inject]')[5].text.replace("'",""))
    wert = soup_zeile.select('[data-itf-inject]')[5].text.replace("'","")

    #die umwandlung in float scheint probleme zu machen, daher wurde sie mal noch ausgelassen

    #Diese einzelne Zeile in ein Minidict schreiben
    #Minidict erstellen

    dict_meldung = {"datum" : datum, "unternehmen":unternehmen, "art":art, "anzahl":anzahl, "wertpapier":wertpapier, "wert":wert}
    liste.append(dict_meldung)

In [11]:
#test

liste=[]

start = "01012018"
ende = "20012018"

site = selenium(start, ende)

suppe(site)
liste

[{'datum': '19.01.2018',
  'unternehmen': 'Ypsomed Holding AG',
  'art': 'Veräusserung',
  'anzahl': '50000',
  'wertpapier': 'Namenaktien',
  'wert': '8800000.00'},
 {'datum': '18.01.2018',
  'unternehmen': 'Swiss Prime Site AG',
  'art': 'Veräusserung',
  'anzahl': '25',
  'wertpapier': 'Namenaktien',
  'wert': '2240.00'},
 {'datum': '18.01.2018',
  'unternehmen': 'nebag ag',
  'art': 'Veräusserung',
  'anzahl': '740',
  'wertpapier': 'Namenaktien',
  'wert': '8140.00'},
 {'datum': '17.01.2018',
  'unternehmen': 'Dottikon ES Holding AG',
  'art': 'Veräusserung',
  'anzahl': '36',
  'wertpapier': 'Namenaktien',
  'wert': '28800.00'},
 {'datum': '17.01.2018',
  'unternehmen': 'KTM Industries AG',
  'art': 'Veräusserung',
  'anzahl': '1414391',
  'wertpapier': 'Inhaberaktien',
  'wert': '8003642.64'},
 {'datum': '17.01.2018',
  'unternehmen': 'Swissquote Group Holding Ltd',
  'art': 'Veräusserung',
  'anzahl': '1000',
  'wertpapier': 'Namenaktien',
  'wert': '46000.00'},
 {'datum': '17.

# Ergebnis anzeigen

Das Ergebnis in einem Panda darstellen und exportieren

In [17]:
#Panda erstellen

datensatz = pd.DataFrame(liste)
datensatz
#datensatz.plot()

# Alles hinter einander ausführen

In [16]:
start = "05012018"
ende = "10012018"

liste = []
site = selenium(start, ende)
suppe(site) 
datensatz = pd.DataFrame(liste)
datensatz

,anzahl,art,datum,unternehmen,wert,wertpapier
0,118186,Erwerb,10.01.2018,Arundel AG,759935.00,Namenaktien
1,24681,Erwerb,10.01.2018,Arundel AG,158698.00,Namenaktien
2,2300000,Veräusserung,10.01.2018,RELIEF THERAPEUTICS Holding SA,19681.20,Namenaktien
3,1830273,Erwerb,09.01.2018,LumX Group Limited,480447.00,Wandelrechte
4,840,Veräusserung,09.01.2018,nebag ag,9240.00,Namenaktien
5,64,Veräusserung,08.01.2018,Partners Group Holding AG,43981.44,Namenaktien
6,20,Erwerb,05.01.2018,Edisun Power Europe AG,848.00,Namenaktien
7,64,Veräusserung,05.01.2018,Partners Group Holding AG,43754.24,Namenaktien
8,12,Veräusserung,27.11.2018,Partners Group Holding AG,8051.76,Namenaktien
9,440,Erwerb,27.11.2018,Ypsomed Holding AG,55000.00,Namenaktien
